In [380]:
import requests
import time
import datetime as dt
import pandas as pd

import hashlib
import hmac
import base64

import json

from pprint import pprint 

from pandas.io.json import json_normalize

import numpy
import logging

In [381]:
# Current time stamp
timestamp = int(time.time())

In [382]:
logging.basicConfig(level=logging.INFO, 
                    format=' %(asctime)s - %(levelname)s - %(message)s')

In [383]:
# Date
today_date = dt.date.today() + dt.timedelta(days=0)
yesterday_date = dt.date.today() + dt.timedelta(days=-1)
seven_days_before_date = dt.date.today() + dt.timedelta(days=-7)

In [384]:
# Shop Parameter
partner_id = 12155
shopid = 23070969
shop_key = 'de5e924b8ed680bc9b22a6c402058154340333ff05641a38069f2856a2f3e24e'

In [385]:
# get_auth_signature
def get_auth_signature(url, payload):
    # API Key
    shopee_secret = shop_key
    # Post Message
    post_message = url + '|' + json.dumps(payload)
    # HMAC-SHA256
    message = bytes(post_message, 'utf-8')
    secret = bytes(shopee_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    auth_signature = hash.hexdigest()
    return auth_signature

In [386]:
# URL
url = 'https://partner.shopeemobile.com/api/v1/items/get'

In [387]:
# get all item
max_page = 200
pagination_entries_per_page = 100
i = 0

try:
    while i <= max_page:
        # start entry
        start_entry = i * pagination_entries_per_page

        # Parameter
        pagination_offset = start_entry

        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "pagination_offset":pagination_offset,
                   "pagination_entries_per_page":pagination_entries_per_page}
        # Auth Signature
        Auth_signature = get_auth_signature(url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_list = json_normalize(parsed['items'])
        # 排除update_time
        selected_column = ['item_id', 'shopid', 'status']
        item_list = item_list[selected_column]

        # 合并
        if i == 0:
            all_item_list = item_list
        else:
            all_item_list = all_item_list.append(item_list)

        i += 1
        logging.info('Retrieving item from page ' + str(i))
except:
    pass

# 去重
logging.info('Remove duplicate records.')
all_item_list = all_item_list.drop_duplicates('item_id').reset_index()
logging.info('Retrieved item list.')
print(all_item_list.head())

 2018-05-20 23:31:43,809 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:44,226 - INFO - Retrieving item from page 1
 2018-05-20 23:31:44,229 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:44,513 - INFO - Retrieving item from page 2
 2018-05-20 23:31:44,517 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:45,190 - INFO - Remove duplicate records.
 2018-05-20 23:31:45,193 - INFO - Retrieved item list.


   index     item_id    shopid  status
0      0  1121446661  23070969  NORMAL
1      1  1121455576  23070969  NORMAL
2      2  1121423066  23070969  NORMAL
3      3   725349781  23070969  NORMAL
4      4   725351767  23070969  NORMAL


In [388]:
# get item detail
item_detail_url = 'https://partner.shopeemobile.com/api/v1/item/get'

In [389]:
item_detail_list = pd.DataFrame()

for key, item in all_item_list.iterrows():
    try:
        item_id = item['item_id']
        logging.info('Now retrieving ' + str(item_id) + ' detail.')
        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "item_id": item_id}
        # Auth Signature
        Auth_signature = get_auth_signature(item_detail_url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(item_detail_url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_detail = json_normalize(parsed['item'])
        item_detail_list = item_detail_list.append(item_detail)
        
        # print(item_detail)
        logging.info('Retrieved ' + str(item_id) + ' detail.')
    except:
        pass
    
# convert timestamp to date
item_detail_list['create_time'] = pd.to_datetime(item_detail_list['create_time'], unit='s').dt.date
item_detail_list['update_time'] = pd.to_datetime(item_detail_list['update_time'], unit='s').dt.date
item_detail_list['days_since_live'] = (today_date - item_detail_list['create_time']).dt.days

 2018-05-20 23:31:45,229 - INFO - Now retrieving 1121446661 detail.
 2018-05-20 23:31:45,234 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:45,724 - INFO - Retrieved 1121446661 detail.
 2018-05-20 23:31:45,725 - INFO - Now retrieving 1121455576 detail.
 2018-05-20 23:31:45,727 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:46,555 - INFO - Retrieved 1121455576 detail.
 2018-05-20 23:31:46,556 - INFO - Now retrieving 1121423066 detail.
 2018-05-20 23:31:46,560 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:47,524 - INFO - Retrieved 1121423066 detail.
 2018-05-20 23:31:47,525 - INFO - Now retrieving 725349781 detail.
 2018-05-20 23:31:47,528 - INFO - Starting new HTTPS connection (1): partner.shopeemobile.com
 2018-05-20 23:31:51,203 - INFO - Retrieved 725349781 detail.
 2018-05-20 23:31:51,204 - INFO - Now retrieving 725351767 detail.
 2018-05-20 23:31:51,208 - INFO - S

In [390]:
item_detail_list

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,sales,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live
0,"[{'is_mandatory': True, 'attribute_id': 7849, ...",5804,0,2018-05-02,TWD,3,Fashion Patented High Quality Anti Slip Crash ...,True,[http://f.shopee.tw/file/a7a43d76b0361451988df...,1121446661,...,0,23070969,NORMAL,5000,2018-05-04,"[{'create_time': 1525277075, 'variation_id': 1...",0,0.10,[],18
0,"[{'is_mandatory': True, 'attribute_id': 7849, ...",5804,0,2018-05-02,TWD,3,Lovely Cartoon Anti Slip Crash Proof Phone Cas...,True,[http://f.shopee.tw/file/43dd056d8d0a1fb574ce1...,1121455576,...,0,23070969,NORMAL,8000,2018-05-02,"[{'create_time': 1525277324, 'variation_id': 1...",1,0.10,[],18
0,"[{'is_mandatory': True, 'attribute_id': 7849, ...",5804,0,2018-05-02,TWD,3,Fashion Patented High Quality Anti Slip Crash ...,True,[http://f.shopee.tw/file/e54a6fd53d6872355c7d3...,1121423066,...,0,23070969,NORMAL,9000,2018-05-02,"[{'create_time': 1525276430, 'variation_id': 1...",3,0.10,[],18
0,"[{'is_mandatory': True, 'attribute_id': 9547, ...",11460,0,2017-11-30,TWD,5,歐美品質 優質線材 適用於各種3孔梅花孔 電源 適配器 充電器 AC線 美規 使用優質銅線製...,False,[http://f.shopee.tw/file/9d77a1606440bcfe3ef12...,725349781,...,0,23070969,NORMAL,1000,2018-03-10,[],25,0.10,[],171
0,"[{'is_mandatory': True, 'attribute_id': 9547, ...",11460,0,2017-11-30,TWD,5,歐美品質 優質線材 適用於各種3孔梅花孔 電源 適配器 充電器 AC線 英規 使用優質銅線製...,False,[http://f.shopee.tw/file/fea0a8aa3e669b532f5eb...,725351767,...,0,23070969,NORMAL,1000,2018-02-28,[],6,0.10,[],171
0,"[{'is_mandatory': True, 'attribute_id': 9547, ...",11460,0,2017-11-30,TWD,5,歐美品質 優質線材 適用於各種3孔梅花孔 電源 適配器 充電器 AC線 歐規 使用優質銅線製...,False,[http://f.shopee.tw/file/42eb3e3452d5f3830720a...,725352772,...,0,23070969,NORMAL,1000,2018-02-28,[],11,0.10,[],171
0,[],5794,2,2017-06-01,TWD,5,廠家直銷高品質超級本適配器19V 1.75A(4.0*1.35) #電源適配器 #適配器 #...,False,[http://f.shopee.tw/file/ca07d670441c20e94b54c...,288208978,...,2,23070969,NORMAL,999,2018-02-28,[],92,0.35,[],353
0,[],5794,1,2017-06-02,TWD,5,Surface 3 平板電腦電源適配器 充電器5.2V2.5A #電源適配器 #適配器 #充...,False,[http://f.shopee.tw/file/7c916db4f2d8876bfe8a7...,289817750,...,1,23070969,NORMAL,1000,2018-02-28,[],63,0.35,[],352
0,[],5794,0,2017-06-02,TWD,5,19V 3.42A筆記本電源適配器 旅行可擕式充電器65W(3.0*1.0小口) #電源適配...,False,[http://f.shopee.tw/file/a71d4c0b2c1e5cd389e04...,289817643,...,2,23070969,NORMAL,1000,2018-02-28,[],44,0.35,[],352
0,[],5794,0,2017-06-02,TWD,5,yoga 900筆記本電源適配器超級本電腦充電器20V2A(USB介面) #電源適配器 #適...,False,[http://f.shopee.tw/file/690654d1793a9ba7c49e2...,289817813,...,1,23070969,NORMAL,1000,2018-02-28,[],35,0.35,[],352


In [391]:
# 选上线超过60天仍然0销量的产品
item_detail_list = item_detail_list[item_detail_list['days_since_live'] >= 60]
item_detail_list = item_detail_list[item_detail_list['sales'] == 0]
item_detail_list = item_detail_list.sort_values(by='days_since_live', ascending=False)
item_detail_list

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,sales,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live
0,[],5794,0,2017-06-02,TWD,3,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口 #電源適配...,False,[http://f.shopee.tw/file/8fa2b388230598f308b1e...,289815382,...,0,23070969,NORMAL,1000,2018-02-02,[],40,0.35,[],352
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,0,23070969,NORMAL,1001,2018-02-28,[],20,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,0,23070969,NORMAL,1001,2018-02-28,[],17,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,0,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,0,23070969,NORMAL,1001,2018-02-28,[],141,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,0,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,0,23070969,NORMAL,1001,2018-02-28,[],14,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,1,2017-11-17,TWD,5,歐美品質 適用於華碩 Ux21A Ux31A Ux32A 電源適配器 4.0mm*1.35m...,False,[http://f.shopee.tw/file/b9d20dce33ed7e7b35116...,692636448,...,0,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,0,23070969,NORMAL,1001,2018-02-28,[],2,0.35,[],184
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,0,23070969,NORMAL,1000,2018-02-28,[],4,0.35,[],184


In [392]:
pd.DataFrame(item_detail_list['attributes'])

,attributes
0,[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,..."


In [393]:
def retrieve_dict_and_convert_to_df_column(dataframe, column_name, selected_key_list, converted_column_name):

    edited_column_list = []

    for index, record in dataframe.iterrows():
        to_edit_column = pd.DataFrame(record[column_name])
        try:
            to_edit_column = to_edit_column[selected_key_list]
            to_edit_column_dict = to_edit_column.to_dict('records')
        except:
            to_edit_column_dict = '[]'
        edited_column_list.append(to_edit_column_dict)
    
    dataframe[converted_column_name] = edited_column_list
    return dataframe

In [394]:
column_name = 'attributes'
selected_key_list = ['attribute_id', 'attribute_value']
converted_column_name = 'edited attributes'

retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,shopid,status,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes
0,[],5794,0,2017-06-02,TWD,3,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口 #電源適配...,False,[http://f.shopee.tw/file/8fa2b388230598f308b1e...,289815382,...,23070969,NORMAL,1000,2018-02-02,[],40,0.35,[],352,[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,23070969,NORMAL,1001,2018-02-28,[],20,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,23070969,NORMAL,1001,2018-02-28,[],17,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,23070969,NORMAL,1001,2018-02-28,[],141,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,23070969,NORMAL,1001,2018-02-28,[],5,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,23070969,NORMAL,1001,2018-02-28,[],14,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,1,2017-11-17,TWD,5,歐美品質 適用於華碩 Ux21A Ux31A Ux32A 電源適配器 4.0mm*1.35m...,False,[http://f.shopee.tw/file/b9d20dce33ed7e7b35116...,692636448,...,23070969,NORMAL,1001,2018-02-28,[],10,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,23070969,NORMAL,1001,2018-02-28,[],2,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,23070969,NORMAL,1000,2018-02-28,[],4,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記..."


In [395]:
column_name = 'variations'
selected_key_list = ['name', 'stock', 'price', 'variation_sku']
converted_column_name = 'edited variations'

retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,status,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes,edited variations
0,[],5794,0,2017-06-02,TWD,3,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口 #電源適配...,False,[http://f.shopee.tw/file/8fa2b388230598f308b1e...,289815382,...,NORMAL,1000,2018-02-02,[],40,0.35,[],352,[],[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,NORMAL,1001,2018-02-28,[],20,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,NORMAL,1001,2018-02-28,[],17,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,NORMAL,1001,2018-02-28,[],10,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,NORMAL,1001,2018-02-28,[],141,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,NORMAL,1001,2018-02-28,[],5,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,NORMAL,1001,2018-02-28,[],14,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,1,2017-11-17,TWD,5,歐美品質 適用於華碩 Ux21A Ux31A Ux32A 電源適配器 4.0mm*1.35m...,False,[http://f.shopee.tw/file/b9d20dce33ed7e7b35116...,692636448,...,NORMAL,1001,2018-02-28,[],10,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,NORMAL,1001,2018-02-28,[],2,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,NORMAL,1000,2018-02-28,[],4,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[]


In [396]:
column_name = 'logistics'
selected_key_list = ['logistic_id', 'enabled']
converted_column_name = 'edited logistics'

retrieve_dict_and_convert_to_df_column(item_detail_list, column_name, selected_key_list, converted_column_name)

,attributes,category_id,cmt_count,create_time,currency,days_to_ship,description,has_variation,images,item_id,...,stock,update_time,variations,views,weight,wholesales,days_since_live,edited attributes,edited variations,edited logistics
0,[],5794,0,2017-06-02,TWD,3,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口 #電源適配...,False,[http://f.shopee.tw/file/8fa2b388230598f308b1e...,289815382,...,1000,2018-02-02,[],40,0.35,[],352,[],[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,692636396,...,1001,2018-02-28,[],20,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,False,[http://f.shopee.tw/file/835be51e6df50f35a7f67...,692636460,...,1001,2018-02-28,[],17,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,692639116,...,1001,2018-02-28,[],10,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,False,[http://f.shopee.tw/file/8e69810052f7edc68eedb...,692640311,...,1001,2018-02-28,[],141,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/2997d787d0579741d5399...,692636465,...,1001,2018-02-28,[],5,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,692636463,...,1001,2018-02-28,[],14,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,1,2017-11-17,TWD,5,歐美品質 適用於華碩 Ux21A Ux31A Ux32A 電源適配器 4.0mm*1.35m...,False,[http://f.shopee.tw/file/b9d20dce33ed7e7b35116...,692636448,...,1001,2018-02-28,[],10,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/310b4d24f9688ff068381...,692636450,...,1001,2018-02-28,[],2,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."
0,"[{'is_mandatory': False, 'attribute_id': 4590,...",5794,0,2017-11-17,TWD,5,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,False,[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,692636455,...,1000,2018-02-28,[],4,0.35,[],184,"[{'attribute_id': 4590, 'attribute_value': '筆記...",[],"[{'logistic_id': 38011, 'enabled': True}, {'lo..."


In [397]:
selected_columns = ["category_id","name","description","price","stock","item_sku","edited variations","images","edited attributes","edited logistics","weight","days_to_ship", 'item_id']
edited_item_detail_list = item_detail_list[selected_columns]
edited_item_detail_list

,category_id,name,description,price,stock,item_sku,edited variations,images,edited attributes,edited logistics,weight,days_to_ship,item_id
0,5794,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口 #電源適配...,400.0,1000,44095986498,[],[http://f.shopee.tw/file/8fa2b388230598f308b1e...,[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,3,289815382
0,5794,歐美品質 適用於華碩 A8 F8 A43S F80 F82 K40 A45 X81 M50 ...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32297003488,[],[http://f.shopee.tw/file/5a41b72bd29474bac4d30...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636396
0,5794,歐美品質 適用於華碩 UX21 UX31 UX31E UX31K UX32 UX42 筆記型...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器 19V 2.37A 45W 3.0mm...,199.0,1001,32297550062,[],[http://f.shopee.tw/file/835be51e6df50f35a7f67...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636460
0,5794,歐美品質 適用華碩 S200E S200 X200T X201E X202E F201E Q...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32486535722,[],[http://f.shopee.tw/file/56c04008c69b47d4c53cd...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692639116
0,5794,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,歐美品質 適用於華碩 A46C M50 X43B A8J K52 筆記型電腦 電源適配器 充...,199.0,1001,32296958243,[],[http://f.shopee.tw/file/8e69810052f7edc68eedb...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692640311
0,5794,歐美品質 適用於華碩 S200 S200E S220 X200T X201E X202E F...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32441829684,[],[http://f.shopee.tw/file/2997d787d0579741d5399...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636465
0,5794,歐美品質 適用於華碩 UX21 UX21E UX31 UX31E UX21A UX31A U...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1001,32441993728,[],[http://f.shopee.tw/file/4e932239e29d0ea46cac9...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636463
0,5794,歐美品質 適用於華碩 Ux21A Ux31A Ux32A 電源適配器 4.0mm*1.35mm,歐美品質 適用於華碩 Ux21A Ux31A Ux32A 電源適配器 4.0mm*1.35m...,279.0,1001,32743847362,[],[http://f.shopee.tw/file/b9d20dce33ed7e7b35116...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636448
0,5794,歐美品質 適用於華碩 M9V R1 S1 S2 S3 S5 A3 A600 F3 X55 A...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,329.0,1001,32743178926,[],[http://f.shopee.tw/file/310b4d24f9688ff068381...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636450
0,5794,歐美品質 適用於華碩 S200 S200E S220 X200T X201E X202E F...,歐美品質 適用於華碩 筆記型電腦 電源適配器 充電器\n\n我司設廠于大陸廣東省廣州市，專職...,249.0,1000,32435638867,[],[http://f.shopee.tw/file/218e48e2be57eaa79a0c5...,"[{'attribute_id': 4590, 'attribute_value': '筆記...","[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,5,692636455


In [398]:
# 测试
test_item_detail_list = edited_item_detail_list[:1]
test_item_detail_list

,category_id,name,description,price,stock,item_sku,edited variations,images,edited attributes,edited logistics,weight,days_to_ship,item_id
0,5794,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口,UX21E UX31E電源適配器19V 2.37A(3.0mm*1.0mm)小口 #電源適配...,400.0,1000,44095986498,[],[http://f.shopee.tw/file/8fa2b388230598f308b1e...,[],"[{'logistic_id': 38011, 'enabled': True}, {'lo...",0.35,3,289815382


In [441]:
def delete_product(delete_product_id):
    # Current time stamp
    timestamp = int(time.time())
    
    # payload
    payload = {
               "item_id": delete_product_id,
               "partner_id": partner_id,
               "shopid": shopid,
               "timestamp": timestamp
               }

    print(payload)
    
    url = 'http://partner.shopeemobile.com/api/v1/item/delete'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)
    # 打印原始Json
    parsed = json.loads(r.text)
    logging.info(parsed)

In [442]:
def add_product(category_id,
               name,
               description,
               price,
               stock,
               item_sku,
               edited_variations,
               images,
               edited_attributes,
               edited_logistics,
               weight,
               days_to_ship
               ):
    # Current time stamp
    timestamp = int(time.time())
    
    # payload
    payload = {
                "category_id": category_id,
                "name": name,
                "description": description,
                "price": price,
                "stock": stock,
                "item_sku": item_sku,
                "images": images,
                "weight": weight,
                "days_to_ship": days_to_ship,
                "partner_id": partner_id,
                "shopid": shopid,
                "timestamp": timestamp
               }
    
    # 判断三个条件是否为空
    if edited_variations != '[]':
        payload['variations'] = edited_variations
        
    if edited_attributes != '[]':
        payload['attributes'] = edited_attributes

    if edited_logistics != '[]':
        payload['logistics'] = edited_logistics
        
    print(payload)

    url = 'http://partner.shopeemobile.com/api/v1/item/add'

    # Auth Signature
    Auth_signature = get_auth_signature(url, payload)
    # HTML Header
    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = requests.post(url, data=payload, headers=headers)  
    # 打印原始Json
    parsed = json.loads(r.text)
    logging.info(parsed)

In [443]:
for index, item in test_item_detail_list.iterrows():
    category_id = item[0]
    name = item[1]
    description = item[2]
    price = item[3]
    stock = item[4]
    item_sku = item[5]
    edited_variations = item[6]
    images = item[7]
    edited_attributes = item[8]
    edited_logistics = item[9]
    weight = item[10]
    days_to_ship = item[11]
    product_id = item[12]
    print(product_id)
    print(type(product_id))
    
    # delete product
    delete_product(product_id)
    
    # add product
    '''
    add_product(category_id,
               name,
               description,
               price,
               stock,
               item_sku,
               edited_variations,
               images,
               edited_attributes,
               edited_logistics,
               weight,
               days_to_ship
               )'''

 2018-05-20 23:43:59,397 - INFO - Starting new HTTP connection (1): partner.test.shopeemobile.com


289815382
<class 'int'>
{'shopid': 23070969, 'item_id': 289815382, 'timestamp': 1526831039, 'partner_id': 12155}


 2018-05-20 23:44:00,349 - INFO - Starting new HTTPS connection (1): partner.test.shopeemobile.com
 2018-05-20 23:44:03,087 - INFO - {'error': 'error_auth'}


In [444]:
# item_detail_list.to_csv("D://item_detail_list.csv", sep=',')
# edited_item_detail_list.to_csv("D://edited_item_detail_list.csv", sep=',')